In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
from tqdm.auto import tqdm
tqdm.pandas()

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

## 1. Processo de Backup e Limpeza de arquivos gerados ontem

In [ ]:

utils.create_backup(first_name = "backup_notifica_diario_" , level=3)
chdir(current_dir)

In [ ]:
from bulletin.utils.clean_up import clear_directories
clear_directories()

## 2. Atualização e Carregamento das Bases de Dados

### 2.1 Carregamento do Notifica e Municípios

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']
municipios.loc[municipios['uf']!='PR','municipio'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
download = False ## <- comentar ´and False´ para baixar novo notifica
load_downloaded = False ## <- comentar ´and False´ para carregar partes baixadas
update = True ## <- comentar ´and False´ para baixar atualizações

if download:
    # Download
    mb = Metabase()
    mb.generate_notifica_query('notifica', where='True', replace=True)
    notifica_parts = mb.download_notificacao('notifica', load=load_downloaded)
    # Leitura
    notifica = Notifica()
    notifica.read(notifica_parts)
    notifica.save('all_notifica_raw', replace=True, compress=True)
#     notifica.load('all_notifica_raw',compress=True)
    ##
    # Normalização
    notifica.normalize()
    notifica.save('notifica', replace=True, compress=False)
############
else:
    # Load 
    notifica = Notifica()
    notifica.databases()
    notifica.load('notifica', compress=False)
    notifica.df = notifica.df.drop_duplicates('id', keep='last')
    
if update:
    mb = Metabase()
    days = 2
    intervalo = f"(data_notificacao >= NOW() - INTERVAL '{days} DAY') or (data_liberacao >= NOW() - INTERVAL '{days} DAY') or (updated_at >= NOW() - INTERVAL '{days} DAY') or (data_coleta >= NOW() - INTERVAL '{days} DAY') or (data_encerramento >= NOW() - INTERVAL '{days} DAY') or (data_cura_obito >= NOW() - INTERVAL '{days} DAY')"
    mb.generate_notifica_query('update_notifica', where=intervalo, replace=True)
    update_notifica_parts = mb.download_notificacao('update_notifica', load=load_downloaded)
    
    update_notifica = Notifica()
    update_notifica.read(update_notifica_parts)
    update_notifica.normalize()
    
    
    update_notifica.df = update_notifica.df.drop_duplicates('id', keep='last')
    analise_dtd = update_notifica.analise_data_diagnostico() ## only in casos confirmados, pd.NaT nos demais

#     novas_notificacoes, atualizacoes_fichas = notifica.update(update_notifica)#, observed_cols=['excluir_ficha','status_notificacao','classificacao_final','data_1o_sintomas','data_diagnostico','evolucao','data_cura_obito','data_encerramento'])
    notifica.update(update_notifica)

    notifica.fix_dtypes()
    notifica.save(replace=True, compress=False)

#     updated_cols = atualizacoes_fichas[['updated_cols','id']].groupby('updated_cols').count().sort_values('id',ascending=False).reset_index()
#     updated_cols.iloc[0:-1,0] += ','
#     updated_cols = list(set(updated_cols.sum()[0].split(',')))

#     for col in updated_cols:
#         print(f"{col}: {atualizacoes_fichas['updated_cols'].str.contains(col).sum()}")
#         display(atualizacoes_fichas.loc[atualizacoes_fichas['updated_cols'].str.contains(col),['id',f'{col}_old',f'{col}_new']])

notifica.df.shape

### 2.2 Carregamento do CC

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}", compress=True)

cc.df.loc[cc.df['sexo']==1, 'sexo'] = 'M'
cc.df.loc[cc.df['sexo']==2, 'sexo'] = 'F'

cc.df.shape

### 2.3 Alterações nos Casos Confirmados

In [ ]:
diff_evolucao = pd.DataFrame()
diff_evolucao = pd.merge(cc.df[['id_notifica','paciente', 'evolucao']], notifica.df[['id', 'evolucao']].rename(columns={'id':'id_notifica'}), on='id_notifica', how='inner', suffixes=['_old','_new'])

In [ ]:
# FICHAS CURA ----> ÓBITOS POR OUTRAS CAUSAS
cura_obitosOutrasCausas = diff_evolucao.loc[(diff_evolucao['evolucao_old'] == 1) & (diff_evolucao['evolucao_new'] == 4)]

for i in tqdm(cura_obitosOutrasCausas['id_notifica']):
    data_cura_obito = notifica.df.loc[notifica.df['id'] == i, 'data_cura_obito'].values[0]
    
    cc.df.loc[cc.df['id_notifica']==i, ('evolucao', 'tipo_encerramento', 'data_cura_obito', 'data_com_evolucao')] = (4, 'E1', data_cura_obito, hoje)

cura_obitosOutrasCausas.shape

In [ ]:
# FICHAS ÓBITO ----> ÓBITOS POR OUTRAS CAUSAS
obito_obitosOutrasCausas = diff_evolucao.loc[(diff_evolucao['evolucao_old'] == 2) & (diff_evolucao['evolucao_new'] == 4)]

temporario_ObitosOutrasCausas = cc.df.loc[cc.df['id_notifica'].isin(obito_obitosOutrasCausas['id_notifica'])]

temporario_ObitosOutrasCausas.to_pickle(join(exclusao_pathfile, f"exclusao_obitos-obitosOutrasCausas.pkl"))
temporario_ObitosOutrasCausas.shape

In [ ]:
# # FICHAS NÃO SE APLICA ----> CURA
# naoAplica_cura = diff_evolucao.loc[(diff_evolucao['evolucao_old'] == 3) & (diff_evolucao['evolucao_new'] == 1)]

# for i in tqdm(naoAplica_cura['id_notifica']):
#     data_cura_obito = notifica.df.loc[notifica.df['id'] == i, 'data_cura_obito'].values[0]
    
#     cc.df.loc[cc.df['id_notifica']==i, ('evolucao', 'tipo_encerramento', 'data_cura_obito', 'data_com_evolucao')] = (1, 'E1', data_cura_obito, hoje)

# naoAplica_cura.shape

In [ ]:
# FICHAS NÃO SE APLICA ----> ÓBITO POR OUTRAS CAUSAS
naoAplica_obitosOutrasCausas = diff_evolucao.loc[(diff_evolucao['evolucao_old'] == 3) & (diff_evolucao['evolucao_new'] == 4)]

for i in tqdm(naoAplica_obitosOutrasCausas['id_notifica']):
    data_cura_obito = notifica.df.loc[notifica.df['id'] == i, 'data_cura_obito'].values[0]
    
    cc.df.loc[cc.df['id_notifica']==i, ('evolucao', 'tipo_encerramento', 'data_cura_obito', 'data_com_evolucao')] = (4, 'E1', data_cura_obito, hoje)

naoAplica_obitosOutrasCausas.shape

In [ ]:
cc.df.shape

In [ ]:
cc.df.groupby(['evolucao'])[['id_notifica']].count()

### 2.4 Ajuste de Casos INATIVOS que voltam à ativa para poderem ser contabilizados

In [ ]:
# MUDANÇA DE VALORES PARA CURITIBA, ficha inativa volta à ativa
notifica.df.loc[(notifica.df['ibge_residencia'] == 410690) & (notifica.df['status_notificacao'] == 3), 'status_notificacao'] = 2

In [ ]:
# FICHAS INATIVAS QUE FOREM REINFECÇAO, DEVEM ENTRAR
notifica.df.loc[notifica.df['reinfeccao'] == True, 'status_notificacao'] = 2

In [ ]:
# ATIVAR FICHAS QUE PRECISAM ENTRAR
list_activate = [1288024, 6159233, 6969412, 4540001]

notifica.df.loc[notifica.df['id'].isin(list_activate), ('status_notificacao', 'excluir_ficha')] = (2, 2)

### 2.5 Filtragem do Notifica COVID-19

In [ ]:
notifica.df = notifica.df.loc[((notifica.df['classificacao_final']==2)&(notifica.df['excluir_ficha']==2)&(notifica.df['status_notificacao'].isin([1,2])))]

notifica.df = pd.merge(notifica.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','macro','rs','mun_resid','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica.df = pd.merge(notifica.df.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

# notifica.df['exame_nome'] = notifica.replace('exame',inplace=False)
notifica.replace('sexo')

notifica.df = notifica.df.loc[((notifica.df['sexo']!='N')&(notifica.df['mun_resid'].notna())&(notifica.df['data_diagnostico'].notna())&(notifica.df['paciente'].str.len() > 5))]

display(notifica.df[['id','evolucao']].groupby(['evolucao']).count())

In [ ]:
notifica_duplicados = notifica.check_duplicates(keep=False)
notifica_duplicados

In [ ]:
# MARCAÇÃO DE FICHAS COMO NÃO DUPLICADAS
list_duplicated_false = [5415309]
notifica.df.loc[notifica.df['id'].isin(list_duplicated_false), 'duplicated'] = False

# INCLUSÃO DE FICHAS DE POSSÍVEIS CASOS DE REINFECÇÃO
notifica.df.loc[notifica.df['reinfeccao'] == True, 'duplicated'] = False

# INSERIR AS FICHAS QUE PRECISAM ENTRAR, independente de duplicidade
notifica.df.loc[notifica.df['id'].isin(list_activate), ('duplicated', 'duplicated_cc')] = (False, False)

In [ ]:
notifica.df['duplicated'] = notifica.df['duplicated'].fillna(False)
notifica.df['duplicated_cc'] = notifica.df['duplicated_cc'].fillna(False)

In [ ]:
notifica.df = notifica.df.loc[~notifica.df['duplicated']]
notifica.df = notifica.df.loc[~notifica.df['duplicated_cc']]

In [ ]:
display(notifica.df[['id','evolucao']].groupby(['evolucao']).count())

### 2.6 Ajuste da Blacklist de Municípios do Notifica

In [ ]:
# BLACKLIST criada para casos confirmados de Fora do País que NINGUÉM consegue ajustar no Notifica
blacklist_ibge = [4396786, 4624182, 5567204]

notifica.df.loc[notifica.df['id'].isin(blacklist_ibge), 'ibge_residencia'] = 999999

# for id_blacklist in blacklist_ibge:
#     try:
#         notifica.df.loc[notifica.df['id'] == id_blacklist, 'ibge_residencia'] = 999999
#     except:
#         pass

## 3. Processo de Exclusões

In [ ]:
try:
    df_exclusao = pd.read_pickle(join(exclusao_pathfile, f"exclusao_notificacoes_{hoje.strftime('%d_%m_%Y')}.pkl"))
    
    df_exclusao.loc[df_exclusao['sexo']==1, 'sexo'] = 'M'
    df_exclusao.loc[df_exclusao['sexo']==2, 'sexo'] = 'F'
except:
    df_exclusao = pd.DataFrame()

## reduzir obitos excluidos
# df_exclusao = df_exclusao.drop(index=df_exclusao.loc[df_exclusao['evolucao']==2].sample(frac=0.8).index)
    
#DEFINIÇÃO DE VARIÁVEIS IMPORTANTES
exclusoes = pd.DataFrame()
exclusoes_casos = pd.DataFrame()
exclusoes_obito = pd.DataFrame()
relatorio_exclusao = []
df_exclusao_len = len(df_exclusao)

In [ ]:
if (df_exclusao_len > 0):
    exclusoes = df_exclusao.copy()
#------------------------------------------------------------------------------------------------------------------

     
# #-------------EXCLUSÃO DE CASOS (E ÓBITOS) ENCAMINHADOS
temporario_ficha = pd.DataFrame()
list_exclusao_casos_obito = pd.read_excel(join(default_input, 'exclusoes', '1-caso_obito.xlsx'))['id_notifica'].to_list()
temporario_ficha = cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_casos_obito)].copy()
if (len(temporario_ficha) > 0):
    exclusoes =  pd.concat([exclusoes, temporario_ficha])




# #-------------EXCLUSÃO DE ÓBITOS ENCAMINHADOS -- > EVOLUCAO = CURA
temporario_cura = pd.DataFrame()
list_exclusao_obito_cura = pd.read_excel(join(default_input, 'exclusoes', '2-obito_cura.xlsx'))['id_notifica'].to_list()
temporario_cura = cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_obito_cura)].copy()
temporario_cura.loc[temporario_cura['evolucao'] == 2, 'evolucao'] = 10
if (len(temporario_cura) > 0):
    exclusoes =  pd.concat([exclusoes, temporario_cura])




# # #-------------EXCLUSÃO DE ÓBITOS ENCAMINHADOS -- > EVOLUCAO = NÃO SE APLICA
temporario_nAplica = pd.DataFrame()
list_exclusao_obito_naplica = pd.read_excel(join(default_input, 'exclusoes', '3-obito_naoSeAplica.xlsx'))['id_notifica'].to_list()
temporario_nAplica = cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_obito_naplica)].copy()
temporario_nAplica.loc[temporario_nAplica['evolucao'] == 2, 'evolucao'] = 10
if (len(temporario_nAplica) > 0):
    exclusoes =  pd.concat([exclusoes, temporario_nAplica])




# # #-------------EXCLUSÃO DE ÓBITOS ENCAMINHADOS -- > EVOLUCAO = ÓBITO POR OUTRAS CAUSAS
try:
    obitosOutrasCausas = pd.read_pickle(join(exclusao_pathfile, f"exclusao_obitos-obitosOutrasCausas.pkl"))
    obitosOutrasCausas = cc.df.loc[cc.df['id_notifica'].isin(obitosOutrasCausas['id_notifica'])].copy()
except:
    obitosOutrasCausas = pd.DataFrame()


temporario_OutrasCausas = pd.DataFrame()

list_exclusao_obito_obitoOutrasCausas = pd.read_excel(join(default_input, 'exclusoes', '4-obito_obitoOutrasCausas.xlsx'))['id_notifica'].to_list()
temporario_OutrasCausas = cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_obito_obitoOutrasCausas)].copy()
temporario_OutrasCausas = pd.concat([temporario_OutrasCausas, obitosOutrasCausas])

list_exclusao_obito_obitoOutrasCausas_final = temporario_OutrasCausas['id_notifica'].tolist()

temporario_OutrasCausas.loc[temporario_OutrasCausas['evolucao'] == 2, 'evolucao'] = 10
if (len(temporario_OutrasCausas) > 0):
    exclusoes =  pd.concat([exclusoes, temporario_OutrasCausas])


# #------------------------------------------------------------------------------------------------------------------



if (len(exclusoes) > 0):
    exclusoes = exclusoes.drop_duplicates('id_notifica', keep='last')

    exclusoes = exclusoes.join(municipios.set_index('ibge'), on='ibge_residencia')
    exclusoes.loc[exclusoes['ibge_residencia']==999999,'mun_resid'] = exclusoes.loc[exclusoes['ibge_residencia']==999999,'pais_residencia']
    
    exclusoes['mun_resid'] = exclusoes['municipio']
    
    exclusoes['motivo_exclusao'] = 'erro de notificação'
    exclusoes['tipo_exclusao'] = 'caso'

    # exclusoes.loc[exclusoes['evolucao']==3,'evolucao'] = 1
    exclusoes.loc[exclusoes['evolucao']==2, 'tipo_exclusao'] = 'caso e óbito'
    exclusoes.loc[exclusoes['evolucao']==10, 'tipo_exclusao'] = 'óbito'
    exclusoes = exclusoes.sort_values('tipo_exclusao', ascending=False)


    exclusoes_casos = exclusoes.loc[exclusoes['tipo_exclusao'] != 'óbito']
    exclusoes_obito = exclusoes.loc[exclusoes['evolucao'].isin([2, 10])]

    exclu_casos_fora = exclusoes.loc[exclusoes['uf_residencia'] != 'PR']
    exclu_obitos_fora = exclusoes.loc[(exclusoes['uf_residencia'] != 'PR') & (exclusoes['evolucao'].isin([2, 10]))]


    relatorio_exclusao = None
    relatorio_exclusao = ( "Um " + exclusoes['tipo_exclusao'] + " confirmado (" + exclusoes['sexo'] + "," + exclusoes['idade'].astype('str') + ") no dia " + exclusoes['data_diagnostico'].dt.strftime('%d/%m/%Y') + " em " + exclusoes['mun_resid'] + " foi excluído por " + exclusoes['motivo_exclusao'] + ".").tolist()
    print(f"TOTAL DE EXLCUSÕES = {len(exclusoes)}. Desse quantitativo, {len(exclusoes_obito)} são óbito")

In [ ]:
if (len(exclusoes_casos) > 0):
    
    # ALTERA OS INDEXES
    cc.df.set_index('id_notifica', inplace=True)
    exclusoes_casos.set_index('id_notifica', inplace=True)
    
    ## REMOÇÃO DOS CASOS DO CC.df
    cc.df = cc.df.drop(index=exclusoes_casos.index)

    # ALTERA OS INDEXES PARA O ESTADO ANTERIOR
    cc.df.reset_index(inplace=True)
    exclusoes_casos.reset_index(inplace=True)
    
    print(f"Foram feitas {len(exclusoes_casos)} exclusões de CASOS. Desse quantitativo, {len(exclusoes_obito)} são óbitos")

### 3.1 Processo de Exclusões de ÓBITOS que tiveram diferentes evoluções

In [ ]:
try:
    cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_obito_cura), ['evolucao']] = (1)
    print(f"Exclusão de ÓBITO -> CURA ({len(list_exclusao_obito_cura)}). Esse valor já está contabilizado no total acima.")
except:
    pass



try:
    cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_obito_naplica), ['evolucao', 'data_cura_obito', 'data_com_evolucao']] = (3, pd.NaT, pd.NaT)
    print(f"Exclusão de ÓBITO -> NÃO SE APLICA ({len(list_exclusao_obito_naplica)}). Esse valor já está contabilizado no total acima.")
except:
    pass



try:
    cc.df.loc[cc.df['id_notifica'].isin(list_exclusao_obito_obitoOutrasCausas_final), ['evolucao', 'data_com_evolucao']] = (4, hoje)
    print(f"Exclusão de ÓBITO -> ÓBITO POR OUTRAS CAUSAS ({len(list_exclusao_obito_obitoOutrasCausas_final)}). Esse valor já está contabilizado no total acima.")
except:
    pass

## 4. Processo de Updates

### 4.1 Alteração de Municípios

In [ ]:
diff_ibge = pd.merge(cc.df[['id_notifica','paciente','sexo','idade','data_diagnostico','ibge_residencia','pais_residencia','evolucao']].replace(cc.tables['pais'].set_index('co_pais').rename(columns={'ds_pais':'pais_residencia'}).to_dict()),notifica.df[['id','ibge_residencia','pais_residencia','evolucao']].rename(columns={'id':'id_notifica'}).replace(notifica.tables['pais'].set_index('co_pais').rename(columns={'ds_pais':'pais_residencia'}).to_dict()),on='id_notifica',how='inner',suffixes=['_old','_new'])
diff_ibge = diff_ibge.loc[diff_ibge['ibge_residencia_old']!=diff_ibge['ibge_residencia_new']]

diff_ibge = pd.merge(diff_ibge.rename(columns={'ibge_residencia_old':'ibge'}),municipios[['ibge','uf','mun_resid']].rename(columns={'uf':'uf_old','mun_resid':'municipio_old'}),on='ibge',how='left').rename(columns={'ibge':'ibge_residencia_old'})
diff_ibge = pd.merge(diff_ibge.rename(columns={'ibge_residencia_new':'ibge'}),municipios[['ibge','uf','mun_resid']].rename(columns={'uf':'uf_new','mun_resid':'municipio_new'}),on='ibge',how='left').rename(columns={'ibge':'ibge_residencia_new'})

diff_ibge.loc[diff_ibge['ibge_residencia_old']==999999,'municipio_old'] = diff_ibge.loc[diff_ibge['ibge_residencia_old']==999999,'pais_residencia_old']
diff_ibge.loc[diff_ibge['ibge_residencia_new']==999999,'municipio_new'] = diff_ibge.loc[diff_ibge['ibge_residencia_new']==999999,'pais_residencia_new']

diff_ibge['tipo_alteracao'] = 'caso'
diff_ibge.loc[diff_ibge['evolucao_old']==3,'evolucao_old'] = 1
diff_ibge.loc[diff_ibge['evolucao_old']==2, 'tipo_alteracao'] = 'caso e óbito'

diff_ibge = diff_ibge.sort_values('evolucao_old', ascending=False)

relatorio_alteracao = ( "Um " + diff_ibge['tipo_alteracao'] + " confirmado (" + diff_ibge['sexo'] + "," + diff_ibge['idade'].astype(str) + ") de " + diff_ibge['municipio_old'] + " foi corrigido para " + diff_ibge['municipio_new'] + ".").tolist()
relatorio_alteracao

In [ ]:
diff_ibge.groupby(['tipo_alteracao'])['tipo_alteracao'].count()

In [ ]:
if ( len(diff_ibge) > 0 ):
    cc.df.set_index('id_notifica', inplace=True)
    cc.df.update(diff_ibge[['id_notifica','ibge_residencia_new','uf_new']].rename(columns={'ibge_residencia_new':'ibge_residencia','uf_new':'uf_residencia'}).set_index('id_notifica'))
    cc.df.reset_index(inplace=True)

### 4.2 Alteração de DO's cadastradas

In [ ]:
do_toSet_none = [1940234,3227237,3407306,3601652,3688233,3765255,4311095,3937987,3304973,3166044,4529450]


notifica.df.loc[notifica.df['numero_do'] == '00000000', 'numero_do'] = None
notifica.df.loc[notifica.df['numero_do'] == '0', 'numero_do'] = None
cc.df.loc[cc.df['numero_do'] == '00000000', 'numero_do'] = None
cc.df.loc[cc.df['numero_do'] == '0', 'numero_do'] = None


notifica.df.set_index('id', inplace=True)
cc.df.set_index('id_notifica', inplace=True)

cc.df.update(notifica.df['numero_do'])

notifica.df.reset_index(inplace=True)
cc.df.reset_index(inplace=True)


for id_do in do_toSet_none:
    try:
        cc.df.loc[cc.df['id_notifica'] == id_do, 'numero_do'] = None
    except:
        pass

In [ ]:

cc.fix_dtypes()
cc.hashes()

## 5. Geração dos novos CASOS

In [ ]:
novos_casos = notifica.df.loc[~(
    (notifica.df['id'].isin(cc.df['id_notifica'])) 
)].copy()


novos_casos = novos_casos.sort_values(['paciente'])
novos_casos['data_com'] = hoje


# Impedir o absurdo que é entrar casos de 2020 ainda
# print(f"CASOS DE 2020 = {len(novos_casos.loc[novos_casos['data_diagnostico'].dt.year == 2020])} (NÃO VÃO ENTRAR)")
# novos_casos.drop(index=novos_casos.loc[novos_casos['data_diagnostico'].dt.year == 2020].index, inplace=True)

print(f"CASOS DE 2020 = {len(novos_casos.loc[novos_casos['data_diagnostico'].dt.year == 2020])}")
print(f"CASOS DE 2021 = {len(novos_casos.loc[novos_casos['data_diagnostico'].dt.year == 2021])}")
print(f"CASOS DE 2022 = {len(novos_casos.loc[novos_casos['data_diagnostico'].dt.year == 2022])}")

novos_casos.shape

novos_casos[['id','data_diagnostico']].groupby('data_diagnostico').count().plot()

In [ ]:
# raise

In [ ]:
dias_apos = [1,2,3,7,14,21,30,60,90]
dias_apos_label = ['hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']

In [ ]:
novos_casos['dias_apos_diagnostico'] = (today - novos_casos['data_diagnostico']).dt.days
novos_casos['periodo_diagnostico'] = [ x for x in np.digitize(novos_casos['dias_apos_diagnostico'],dias_apos,right=False)]
periodo_novos_casos = novos_casos[['id','periodo_diagnostico']].groupby('periodo_diagnostico').count().rename(columns={'id':'qtde'})
periodo_novos_casos['periodo'] = [ dias_apos_label[x] for x in periodo_novos_casos.index ]
periodo_novos_casos[['periodo','qtde']]

## 6. Geração dos novos ÓBITOS

In [ ]:
obitos_notifica = notifica.df.loc[(notifica.df['evolucao']==2)]
obitos_casos = cc.df.loc[(cc.df['evolucao']==2)]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['id'].isin(obitos_casos['id_notifica']))
)].copy()

# print('novos obitos ',len(novos_obitos))
novos_obitos.to_excel(join(default_output,'correcoes','qualificar_esses_obitos.xlsx'))
print('obitos_sem_data_cura_obito ',len(novos_obitos.loc[novos_obitos['data_cura_obito'].isna()]))

novos_obitos.loc[novos_obitos['data_cura_obito'].isna()].to_excel(join(default_output,'correcoes','obitos_sem_data_cura_obito.xlsx'))
novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]


novos_obitos = novos_obitos.loc[novos_obitos['id'].isin(cc.df['id_notifica'].tolist() + novos_casos['id'].tolist())]

# print(len(novos_obitos))

novos_obitos = novos_obitos.sort_values(['paciente'])
novos_obitos['data_com_evolucao'] = hoje



## Descomentar essas células quando precisa limitar a divulgação de óbitos novos diário, 
## solicitado pela chefia imediata.
# novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'] >= pd.to_datetime('2021-01-01')]
# novos_obitos = novos_obitos.sample(frac=0.5,weights=novos_obitos.groupby('data_cura_obito')['data_cura_obito'].transform('count'))



novos_obitos = novos_obitos.drop_duplicates('id', keep='last')

print('novos obitos = ', len(novos_obitos))

novos_obitos[['id','data_cura_obito']].groupby('data_cura_obito').count().plot()

In [ ]:
# factor_day = float(0.18 + (hoje.day / 1000) + 0.022)
factor_day = 0.5
print(factor_day)

In [ ]:
novos_casos_obitos = novos_casos.loc[novos_casos['id'].isin(novos_obitos['id'])]
novos_casos = novos_casos.loc[~novos_casos['id'].isin(novos_obitos['id'])]


# if ((hoje.day_name() == 'Saturday') or (hoje.day_name() == 'Sunday')):
#     novos_casos = novos_casos.loc[novos_casos['data_diagnostico']>pd.to_datetime('2021-01-01')]
#     novos_casos = novos_casos.sample(frac=factor_wkns,weights=novos_casos.groupby('data_diagnostico')['data_diagnostico'].transform('count'))
# else:
#     novos_casos = novos_casos.sample(frac=factor_day,weights=novos_casos.groupby('data_diagnostico')['data_diagnostico'].transform('count'))

# novos_casos = novos_casos.loc[novos_casos['data_diagnostico'] >= pd.to_datetime('2021-01-01')]

novos_casos_ate_7_dias = novos_casos.loc[novos_casos['periodo_diagnostico'] <= 4 ]
novos_casos = novos_casos.loc[~novos_casos['id'].isin(novos_casos_ate_7_dias['id'])]
print(novos_casos_ate_7_dias.shape)



## Descomentar essa célula quando precisa limitar a divulgação de casos novos diário, 
## solicitado pela chefia imediata.
# novos_casos = novos_casos.sample(frac=factor_day,weights=novos_casos.groupby('data_diagnostico')['data_diagnostico'].transform('count'))

print(novos_casos.shape)

novos_casos = novos_casos.append(novos_casos_obitos).append(novos_casos_ate_7_dias)

novos_casos = novos_casos.drop_duplicates('id', keep='last')

novos_casos[['id','data_diagnostico']].groupby('data_diagnostico').count().plot()

In [ ]:
novos_casos.shape

## 7. Geração dos novos RECUPERADOS

In [ ]:
recuperados_notifica = notifica.df.loc[(notifica.df['evolucao']==1)]
recuperados_casos = cc.df.loc[(cc.df['evolucao'].isin([1,2]))]

novos_recuperados = recuperados_notifica.loc[~(
    (recuperados_notifica['id'].isin(recuperados_casos['id_notifica']))
)].copy()


novos_recuperados = novos_recuperados.loc[novos_recuperados['data_cura_obito'].notna()]

novos_recuperados = novos_recuperados.loc[novos_recuperados['id'].isin(cc.df['id_notifica'].tolist() + novos_casos['id'].tolist())]

novos_recuperados = novos_recuperados.sort_values(['paciente'])
novos_recuperados['data_com_evolucao'] = hoje

novos_recuperados = novos_recuperados.loc[~novos_recuperados['id'].isin(cc.df.loc[(cc.df['evolucao']==2),'id_notifica'])]

print(len(novos_recuperados))
novos_recuperados[['id','data_cura_obito']].groupby('data_cura_obito').count().plot()

In [ ]:
# GERAÇÃO DOS RECUPERADOS AUTOMÁTICOS

novos_recuperados_automatico = pd.DataFrame()
limit_day_active = pd.to_datetime(today - timedelta(60))
novos_recuperados_automatico = cc.df.loc[(cc.df['id_notifica'] > 0) & ~(cc.df['id_notifica'].isin(novos_obitos['id'])) & (cc.df['data_diagnostico'] < limit_day_active) & (cc.df['evolucao'] == 3) & ~(cc.df['id_notifica'].isin(novos_recuperados['id']))].copy()
novos_recuperados_automatico[['evolucao', 'tipo_encerramento', 'data_cura_obito', 'data_com_evolucao']] = (1, 'E3', hoje, hoje)
novos_recuperados_automatico.shape

## 8. Geração do Relatório e Log de Modificação

In [ ]:
# GERA ARQUIVO DE ALTERAÇÕES FEITAS
writer = pd.ExcelWriter(join(default_output,'relatorios', f"alteracoes_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

if len(novos_casos) != 0: novos_casos.to_excel(writer,'novos_casos')
if len(novos_obitos) != 0: novos_obitos.to_excel(writer,'novos_obitos')
if len(novos_recuperados) != 0: novos_recuperados.to_excel(writer,'novos_recuperados')
if len(novos_recuperados_automatico) != 0: novos_recuperados_automatico.to_excel(writer,'novos_recuperados_automatico')
if len(diff_ibge) != 0: diff_ibge.to_excel(writer,'alteracao_municipio')
if len(exclusoes_casos) != 0: exclusoes_casos.to_excel(writer,'exclusoes_casos') ## excluiu independente da evolução
if len(exclusoes_obito) != 0: exclusoes_obito.to_excel(writer,'exclusoes_obitos') ## mudou evolução apenas, manteve o caso

writer.save()
writer.close()

In [ ]:
casos_confirmados = pd.merge(cc.df.rename(columns={'ibge_residencia':'ibge'}),municipios,how='left',on='ibge').rename(columns={'ibge':'ibge_residencia'}).copy()
date_parser = lambda x: x
relatorio_file = join(default_output,'relatorios',f"relatorio_{(datetime.today().strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
relatorio_file

In [ ]:
casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de óbitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de óbitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de óbitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_casosPR_group = novos_casosPR[['id','municipio']].groupby(by='municipio').count().rename(columns={'id':'qtde'}).reset_index().sort_values(['qtde','municipio'],ascending=False)
novos_obitosPR_group = novos_obitosPR[['id','municipio']].groupby(by='municipio').count().rename(columns={'id':'qtde'}).reset_index().sort_values(['qtde','municipio'],ascending=False)

data_retroativos = ontem - timedelta(31)

retroativos = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) <= data_retroativos)].sort_values(by='data_diagnostico')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_diagnostico'].apply(date_parser) <= anteontem)].sort_values(by='data_diagnostico')
casos_hoje = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) > anteontem)].sort_values(by='data_diagnostico')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    # for mun, df in novos_casosPR_group.iterrows():
    #     relatorio.write(f"{df['qtde']:,} {df['municipio']}\n".replace(',','.'))

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))

    relatorio.write(f"{len(novos_obitosPR):,} Óbito{'s' if len(novos_obitosPR) > 1 else ''} residente{'s' if len(novos_obitosPR) > 1 else ''} do PR:\n".replace(',','.'))

    for mun, df in novos_obitosPR_group.iterrows():
        relatorio.write(f"{df['qtde']:,} {df['municipio']}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))


#RELATÓRIO DE EXCLUSÕES
    relatorio.write(f"*Relatório de Exclusões:*\n")
    try:
        if len(relatorio_exclusao) > 0:
            relatorio.write(f"{len(exclusoes_casos)} caso{'s' if len(exclusoes_casos) > 1 else ''} ")
            if (len(exclusoes_obito) > 0):
                 relatorio.write(f"e {len(exclusoes_obito)} óbito{'s' if len(exclusoes_obito) > 1 else ''} ")

            relatorio.write(f"{'foram' if (len(exclusoes_casos) + len(exclusoes_obito)) > 1 else 'foi'} excluído{'s' if (len(exclusoes_casos) + len(exclusoes_obito)) > 1 else ''}.\n")

            if (len(exclu_casos_fora) > 0):
                relatorio.write(f"Desse quantitativo, ")
                relatorio.write(f"{len(exclu_casos_fora)} caso{'s' if len(exclu_casos_fora) > 1 else ''} ")
                if (len(exclu_obitos_fora) > 0):
                     relatorio.write(f"e {len(exclu_obitos_fora)} óbito{'s' if len(exclu_obitos_fora) > 1 else ''} ")

                relatorio.write(f"{'são' if (len(exclu_casos_fora) + len(exclu_obitos_fora)) > 1 else 'é'} de Fora do Paraná.\n")                

            relatorio.write('\n')
            for row in relatorio_exclusao:
                relatorio.write(row+'\n')
            relatorio.write('\n')
    except:
        relatorio.write(f"Nenhuma exclusão foi feita até o momento da publicação deste relatório.\n\n")

    

#RELATÓRIO DE ALTERAÇÃO DE MUNICÍPIO 
    relatorio.write(f"*Relatório de Correções de Municípios:*\n")
    if len(relatorio_alteracao) > 0:  
        diff_ibge_obito = diff_ibge.loc[diff_ibge['evolucao_old']==2]
        
        relatorio.write(f"{len(diff_ibge)} caso{'s' if len(diff_ibge) > 1 else ''} ")
        if (len(diff_ibge_obito) > 0):
            relatorio.write(f"e {len(diff_ibge_obito)} óbito{'s' if len(diff_ibge_obito) > 1 else ''} ")
        
        relatorio.write(f"{'foram' if (len(diff_ibge) + len(diff_ibge_obito)) > 1 else 'foi'} corrigido{'s' if (len(diff_ibge) + len(diff_ibge_obito)) > 1 else ''}.\n")
            
        
        pr_pr = diff_ibge.loc[(diff_ibge['uf_old']=='PR')&(diff_ibge['uf_new']=='PR')]
        pr_pr_obitos = pr_pr.loc[(pr_pr['evolucao_old']==2)]
        
        pr_fora = diff_ibge.loc[(diff_ibge['uf_old']=='PR')&(diff_ibge['uf_new']!='PR')]
        pr_fora_obitos = pr_fora.loc[(pr_fora['evolucao_old']==2)]
        
        fora_pr = diff_ibge.loc[(diff_ibge['uf_old']!='PR')&(diff_ibge['uf_new']=='PR')]
        fora_pr_obitos = fora_pr.loc[(fora_pr['evolucao_old']==2)]
        
        fora_fora = diff_ibge.loc[(diff_ibge['uf_old']!='PR')&(diff_ibge['uf_new']!='PR')]
        fora_fora_obitos = fora_fora.loc[(fora_fora['evolucao_old']==2)]
        
        if (len(pr_pr) > 0):
            relatorio.write(f"{len(pr_pr)} caso{'s' if len(pr_pr) > 1 else ''} ")
            if (len(pr_pr_obitos) > 0):
                relatorio.write(f"e {len(pr_pr_obitos)} óbito{'s' if len(pr_pr_obitos) > 1 else ''} residente{'s' if (len(pr_pr) + len(pr_pr_obitos)) > 1 else ''} alterado{'s' if (len(pr_pr) + len(pr_pr_obitos)) > 1 else ''} entre municípios do Paraná.\n")
            else:
                relatorio.write(f"residente{'s' if (len(pr_pr) + len(pr_pr_obitos)) > 1 else ''} alterado{'s' if (len(pr_pr) + len(pr_pr_obitos)) > 1 else ''} entre municípios do Paraná.\n")
        
        
        if (len(pr_fora) > 0):
            relatorio.write(f"{len(pr_fora)} caso{'s' if len(pr_fora) > 1 else ''} ")
            if (len(pr_fora_obitos) > 0):
                relatorio.write(f"e {len(pr_fora_obitos)} óbito{'s' if len(pr_fora_obitos) > 1 else ''} residente{'s' if (len(pr_fora) + len(pr_fora_obitos)) > 1 else ''} do Paraná alterado{'s' if (len(pr_fora) + len(pr_fora_obitos)) > 1 else ''} para Fora do Estado.\n")
            else:
                relatorio.write(f"residente{'s' if (len(pr_fora) + len(pr_fora_obitos)) > 1 else ''} do Paraná alterado{'s' if (len(pr_fora) + len(pr_fora_obitos)) > 1 else ''} para Fora do Estado.\n")
        
        
        if (len(fora_pr) > 0):
            relatorio.write(f"{len(fora_pr)} caso{'s' if len(fora_pr) > 1 else ''} ")
            if (len(fora_pr_obitos) > 0):
                relatorio.write(f"e {len(fora_pr_obitos)} óbito{'s' if len(fora_pr_obitos) > 1 else ''} residente{'s' if (len(fora_pr) + len(fora_pr_obitos)) > 1 else ''} de Fora do Estado alterado{'s' if (len(fora_pr) + len(fora_pr_obitos)) > 1 else ''} para o Paraná.\n")
            else:
                relatorio.write(f"residente{'s' if (len(fora_pr) + len(fora_pr_obitos)) > 1 else ''} de Fora do Estado alterado{'s' if (len(fora_pr) + len(fora_pr_obitos)) > 1 else ''} para o Paraná.\n")
                
        
        if (len(fora_fora) > 0):
            relatorio.write(f"{len(fora_fora)} caso{'s' if len(fora_fora) > 1 else ''} ")
            if (len(fora_fora_obitos) > 0):
                relatorio.write(f"e {len(fora_fora_obitos)} óbito{'s' if len(fora_fora_obitos) > 1 else ''} residente{'s' if (len(fora_fora) + len(fora_fora_obitos)) > 1 else ''} de Fora do Estado alterado{'s' if (len(fora_fora) + len(fora_fora_obitos)) > 1 else ''} entre si.\n")
            else:
                relatorio.write(f"residente{'s' if (len(fora_fora) + len(fora_fora_obitos)) > 1 else ''} de Fora do Estado alterado{'s' if (len(fora_fora) + len(fora_fora_obitos)) > 1 else ''} entre si.\n")
        
        relatorio.write('\n')               
        for row in relatorio_alteracao:
            relatorio.write(row+'\n')
        relatorio.write('\n')
    
    else:
        relatorio.write(f"Nenhuma correção de município foi feita até o momento da publicação deste relatório.\n\n")

        
            
    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{int(row['idade'])}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(int(row['rs'])).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:
        #IMPRESSÃO DOS CASOS
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR:\n".replace(',','.'))

        try:
            relatorio.write(f"{len(retroativos):,} caso{'s' if len(retroativos) > 1 else ''} retroativo{'s' if len(retroativos) > 1 else ''} confirmado{'s' if len(retroativos) > 1 else ''} no período de {retroativos.iloc[0]['data_diagnostico'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_diagnostico'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        except:
            relatorio.write(f"Sem novos casos retroativos.\n")
        
        try:
            relatorio.write(f"{len(last2weeks):,} novo{'s' if len(last2weeks) > 1 else ''} caso{'s' if len(last2weeks) > 1 else ''} confirmado{'s' if len(last2weeks) > 1 else ''} no período de {last2weeks.iloc[0]['data_diagnostico'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_diagnostico'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        except:
            relatorio.write(f"Sem novos casos ocorridos entre {(data_retroativos - timedelta(1)).strftime('%d/%m/%Y')} e {ontem.strftime('%d/%m/%Y')}.\n")
        
        try:
            relatorio.write(f"{len(casos_hoje):,} novo{'s' if len(casos_hoje) > 1 else ''} caso{'s' if len(casos_hoje) > 1 else ''} confirmado{'s' if len(casos_hoje) > 1 else ''} nas últimas 24 horas.\n".replace(',','.'))
        except:
            relatorio.write(f"Sem novos casos ocorridos nas últimas 24 horas.\n")
        
        relatorio.write('\n')
        
        #casos por mês e dia
        try:
            novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_diagnostico'].month, axis=1)
            novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_diagnostico'].year, axis=1)
            relatorio.write('Novos casos por meses:\n')
            for group, value in novos_casosPR.groupby(by=['year','month']):
                relatorio.write(f"{static.meses[int(group[1])-1]}/{group[0]}: {len(value)}\n")
            relatorio.write('\n')

            relatorio.write('Novos casos por dia:\n')
            for group, value in novos_casosPR.groupby(by='data_diagnostico'):
                relatorio.write(f"{group.strftime('%d/%m/%Y')}\t{len(value)}\n")
            relatorio.write('\n')
        except:
            pass
        
        
        
        #IMPRESSÃO DOS ÓBITOS
        relatorio.write(f"{len(novos_obitosPR):,} novos óbitos residentes divulgados no PR:\n".replace(',','.'))

        try:
            relatorio.write(f"{len(obitos_retroativos):,} óbito{'s' if len(obitos_retroativos) > 1 else ''} retroativo{'s' if len(obitos_retroativos) > 1 else ''} ocorrido{'s' if len(obitos_retroativos) > 1 else ''} no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        except:
            relatorio.write(f"Sem novos óbitos retroativos.\n")
        
        if len(obitos_last2weeks) > 0:
            relatorio.write(f"{len(obitos_last2weeks):,} novo{'s' if len(obitos_last2weeks) > 1 else ''} óbito{'s' if len(obitos_last2weeks) > 1 else ''} ocorrido{'s' if len(obitos_last2weeks) > 1 else ''} entre {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        else:
            relatorio.write(f"Sem novos óbitos ocorridos entre {(data_retroativos - timedelta(1)).strftime('%d/%m/%Y')} e {ontem.strftime('%d/%m/%Y')}.\n")
        
        if len(obitos_hoje) > 0:
            relatorio.write(f"{len(obitos_hoje):,} novo{'s' if len(obitos_hoje) > 1 else ''} óbito{'s' if len(obitos_hoje) > 1 else ''} ocorrido{'s' if len(obitos_hoje) > 1 else ''} nas últimas 24 horas.\n".replace(',','.'))
        else:
            relatorio.write(f"Sem novos óbitos ocorridos nas últimas 24 horas.\n")
        
        relatorio.write('\n')
        
        #óbitos por mês e dia
        try:
            relatorio.write('Novos óbitos por meses:\n')
            novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
            novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
            for group, value in novos_obitosPR.groupby(by=['year','month']):
                relatorio.write(f"{static.meses[int(group[1])-1]}/{group[0]}: {len(value)}\n")
            relatorio.write('\n')

            relatorio.write('Novos óbitos por dia:\n')
            for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
                relatorio.write(f"{group.strftime('%d/%m/%Y')}\t{len(value)}\n")
            relatorio.write('\n')
        except:
            pass
        
        #--------------RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group.iterrows():
            obito = obitos['qtde']
            obitos_list.append(obito)
            munic.append(obitos['municipio'])             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios: ")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")
        relatorio.write(f".\n")

In [ ]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

In [ ]:
# raise

## 9. Alterações necessárias no CC e TB Datas

In [ ]:
tb_datas = pd.read_excel(join(default_input,'TB_Datas.xlsx'))
tb_datas.set_index('Data',inplace=True)
tb_datas.loc[today.strftime('%d/%m/%Y'), ['Obitos','Casos']] = (len(novos_obitosPR),len(novos_casosPR))
tb_datas['Identificação'] = [ x for x in range(1,len(tb_datas)+1) ]
tb_datas = tb_datas.astype(int)
tb_datas.reset_index(inplace=True)
tb_datas.set_index('Identificação',inplace=True)
tb_datas.to_excel(join(default_input,'TB_Datas.xlsx'))
display(tb_datas)

In [ ]:
# cc.load(compress=True)
# cc.fix_dtypes()
# cc.hashes()
casos_confirmados = cc.df
casos_confirmados.set_index('id_notifica',inplace=True)
# cc.df.update(diff_ibge[['id_notifica','ibge_residencia_new','uf_new']].rename(columns={'ibge_residencia_new':'ibge_residencia','uf_new':'uf_residencia'}).set_index('id_notifica'))
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

In [ ]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'data_com': 'data_comunicacao'
})

novos_casos['uf_residencia'] = novos_casos['uf']
novos_casos['data_com_evolucao'] = pd.NaT
novos_casos['acc_merge_notifica'] = None
novos_casos['tipo_encerramento'] = None
novos_casos['evolucao'] = 3

# novos_casos['exame'] = None !! ??

# novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3

novos_casos['identificacao'] = list(range(casos_confirmados.iloc[-1,0] + 1,casos_confirmados.iloc[-1,0] + 1 + len(novos_casos)))

novos_casos.set_index('id_notifica',inplace=True)
novos_casos = novos_casos[[ col for col in casos_confirmados.columns if col in novos_casos.columns ]]

casos_confirmados = casos_confirmados.append(novos_casos)

In [ ]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica'
})

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2
novos_obitos['uf_residencia'] = novos_obitos['uf']

novos_obitos = novos_obitos[['id_notifica','evolucao','data_cura_obito','data_com_evolucao','numero_do']]
# novos_obitos = novos_obitos[[ col for col in casos_confirmados.columns if col in novos_obitos.columns ]]

novos_obitos['tipo_encerramento'] = 'E1'

novos_obitos.set_index('id_notifica',inplace=True)
casos_confirmados.update(novos_obitos)

# novos_obitos

In [ ]:
novos_recuperados = novos_recuperados.rename(columns={
    'id': 'id_notifica'
})

novos_recuperados['data_com_evolucao'] = pd.to_datetime(date.today())
novos_recuperados['evolucao'] = 1
novos_recuperados['uf_residencia'] = novos_recuperados['uf']
novos_recuperados['tipo_encerramento'] = 'E1'

novos_recuperados = novos_recuperados[['id_notifica','evolucao','data_cura_obito','data_com_evolucao']]
# novos_recuperados = novos_recuperados[[ col for col in casos_confirmados.columns if col in novos_recuperados.columns ]]

casos_confirmados.update(novos_recuperados.set_index('id_notifica'))

if len(novos_recuperados_automatico) != 0:
    novos_recuperados_automatico = novos_recuperados_automatico[['id_notifica','uf_residencia','ibge_residencia','evolucao','data_cura_obito','data_com_evolucao']]
    casos_confirmados.update(novos_recuperados_automatico.set_index('id_notifica'))

## 10. Processo Final

In [ ]:
cc.df = casos_confirmados.reset_index()

### 10.1 Ajustes necessários

In [ ]:
cc.df['reinfeccao'] = cc.df['reinfeccao'].fillna(False)
cc.df['reinfeccao_ids'] = cc.df['reinfeccao_ids'].fillna(-1)


cc.df.loc[cc.df['sexo'] == 'Masculino', 'sexo'] = 'M'
cc.df.loc[cc.df['sexo'] == 'Feminino', 'sexo'] = 'F'
cc.df.loc[cc.df['sexo']==1, 'sexo'] = 'M'
cc.df.loc[cc.df['sexo']==2, 'sexo'] = 'F'

In [ ]:
cc.df.groupby('sexo')[['id_notifica']].count()

In [ ]:
cc.df.groupby('reinfeccao')[['id_notifica']].count()

### 10.2 Atualização de algumas colunas

In [ ]:
# updateable_columns = pd.read_csv(join(default_input, "updateable_columns.csv"))
updateable_columns = cc.schema.loc[cc.schema['updateable']==1]
updateable_columns_list = updateable_columns['column'].to_list()

notifica.df.set_index('id', inplace=True)
cc.df.set_index('id_notifica', inplace=True)

cc.df.update(notifica.df[updateable_columns_list])

notifica.df.reset_index(inplace=True)
cc.df.reset_index(inplace=True)

### 10.3 Normalizar valores e cálculo de hashes

In [ ]:
cc.fix_dtypes()
cc.normalize()
cc.hashes()

### 10.4 Métricas de avaliações finais

In [ ]:
print(cc.df.shape)
# casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()
cc.df.groupby(['evolucao'])[['identificacao']].count()

### 10.5 Salva a base de dados de hoje

In [ ]:
cc.save(f"cc_{hoje.strftime('%d_%m_%Y')}", replace=True, compress=True)